In [1]:
# Code to parallel process OTAR json file to extract sentences and entities
# (c) EMBL-EBI, June 2019
#
# Started: 9 Septmember  2019
# Updated: 9 Septmember  2019

_author_ = 'Santosh Tirunagari'

import os
import pandas as pd
import glob
import json

import sys

import multiprocessing

import numpy as np
import re

from nltk.tokenize import wordpunct_tokenize

In [2]:
colNames = ('text', 'ner')

file_path = '/nfs/gns/literature/Santosh_Tirunagari/EBI standard Dataset/'
all_files = glob.glob(file_path+'*fulltext_batch*')
result_folder = '/nfs/gns/literature/Santosh_Tirunagari/EBI standard Dataset/NER/'

In [5]:
import csv
with open('/nfs/gns/literature/Santosh_Tirunagari/EBI standard Dataset/NER/list_pmc_ids.csv','w') as f1:
    writer=csv.writer(f1, delimiter='\t',lineterminator='\n',)
    
    for files in all_files:
        with open(files) as json_file_ner_rel:
            json_data = json.loads(json_file_ner_rel.read())
            for articles in json_data:
                pmc_id = articles #json_data[articles]
                writer.writerow([pmc_id])
                

In [ ]:
import random
 
def split_file(file,train,devel,test,percentage=0.70,isShuffle=True,seed=123):
"""Splits a file in 3 given the `percentage` to go in the large file."""
random.seed(seed)
with open(file, 'r',encoding="utf-8") as fin:
 
    nLines = sum(1 for line in fin)
    fin.seek(0)
 
    nTrain = int(nLines*percentage) 
    
    nValid = (nLines - nTrain)/2
    
    nTest = nLines - nValid
 
    i = 0
    for line in fin:
        r = random.random() if isShuffle else 0 # so that always evaluated to true when not isShuffle
        if (i < nTrain and r < percentage) or (nLines - i > nValid):
            foutBig.write(line)
            i += 1
        else:
            foutSmall.write(line)


In [19]:
import math
import random
file = '/nfs/gns/literature/Santosh_Tirunagari/EBI standard Dataset/NER/list_pmc_ids.csv'
percentage=0.70
with open(file, 'r',encoding="utf-8") as fin:
 
    nLines = sum(1 for line in fin)
    fin.seek(0)
 
    nTrain = int(nLines*percentage) 
    
    nValid = math.floor((nLines - nTrain)/2)
    
    nTest = nLines - (nTrain+nValid)

nTrain, nValid, nTest    

(125, 27, 28)

In [30]:
r = random.random() 
r

0.5172811423030917

In [ ]:
r = random.random() 
if isShuffle else 0 # so that always evaluated to true when not isShuffle
    if (i < nTrain and r < percentage) or (nLines - i > nValid):
#         foutBig.write(line)
            
        i += 1
    else:
        foutSmall.write(line)


In [3]:
with open(all_files[0]) as json_file_ner_rel:
    json_data = json.loads(json_file_ner_rel.read())

In [4]:
import csv
with open('/nfs/gns/literature/Santosh_Tirunagari/EBI standard Dataset/NER/annot.csv','w') as f1:
    writer=csv.writer(f1, delimiter='\t',lineterminator='\n',)

    for articles in json_data:
        pmc_id = articles #json_data[articles]
    #     print(pmc_id)
        for each_annotation in json_data[articles]['annotations']:
#             break
    #         print(each_annotation)
              

            if each_annotation['ner'] != None:
                text = each_annotation['sent'].encode('utf-8').decode('utf-8')
                ner = each_annotation['ner']


                split_text = wordpunct_tokenize(text)
                arr = ['O']*len(split_text)
                ner_dict = dict(zip(split_text,arr))

                ner_dict_ = {}

                for each_tag in ner:
                    token_list = wordpunct_tokenize(each_tag[2])
                    ner_list = wordpunct_tokenize(each_tag[3])

                    if(len(token_list) > len(ner_list)):
                        ner_list = len(token_list) * ner_list

                    for i in range(0,len(ner_list)):
                        if(i==0):
                            ner_list[i] = 'B-'+ner_list[i]
                        else:
                            ner_list[i] = 'I-'+ner_list[i]


                    ner_dict_.update(dict(zip(token_list,ner_list)))
                    

                ner_dict.update(ner_dict_)
                    
                for each_word in split_text:
                    row = [each_word,ner_dict[each_word]]
                    writer.writerow(row)
                writer.writerow('\n')    


        
            